In [ ]:
# Generating samples of game state with GPT-4o
# Instead of simulating every step, we ask GPT to generate a probable game state,
# given that guesser is very smart and eventually wins

import asyncio
import pickle

import pandas as pd
%reload_ext autoreload
%autoreload 2
%autoreload now
import logging

from data_collection.generate_snapshot import generate_snapshot
from data_collection.semaphore import set_semaphore

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from dotenv import load_dotenv
from openai import AsyncOpenAI

load_dotenv()

In [ ]:
set_semaphore("gpt-4", 100)

In [ ]:
client = AsyncOpenAI()

In [ ]:
# {
#         "things": 100,
#         "city": 40,
#         "country": 20,
#         "landmark": 40,
#     }

In [ ]:
keywords_by_category = pickle.load(open("../keywords_by_category.pkl", "rb"))
all_dfs = pd.concat(keywords_by_category.values())
keywords_flat = list(all_dfs.keyword)

In [ ]:
# generated = await generate_snapshot(client, "ice tea")
all_generated = await asyncio.gather(*[generate_snapshot(client, keyword) for keyword in keywords_flat])

In [ ]:
total_generated = [generated for generated_list in all_generated for generated in generated_list]

In [ ]:
pickle.dump(total_generated, open("dataset_17476_from_snapshots.pkl", "wb"))

In [ ]:
for g in total_generated:
    if g['type'] == "guess_from_snapshot":
        keyword = g['response'].split("\n")[-1].strip()
        prompt_lines = g['prompt'].split("\n")
        # Find indexes of line that contain keyword:
        keyword_indexes = [i for i, line in enumerate(prompt_lines) if f"Your guess: {keyword}".lower().strip() in line.lower()]
        if not keyword_indexes and keyword.endswith("s"):
            keyword_indexes = [i for i, line in enumerate(prompt_lines) if f"Your guess: {keyword[:-1]}".lower().strip() in line.lower()]
        if keyword_indexes:
            print(g['prompt'])
            print("!!!!!!!!", keyword)
        # g['prompt'] = "\n".join([line for i, line in enumerate(prompt_lines) if i not in keyword_indexes])

In [ ]:
for g in total_generated:
    if g['type'] == "questions_candidates_from_snapshot":
        new_response = g['response'].split("Question: ")[1].split("Answer")[0].strip()
        g['response'] = new_response

In [ ]:
pickle.dump(total_generated, open("dataset_17476_from_snapshots_fixed.pkl", "wb"))